In [1]:
#5.1
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.1621,  0.3344, -0.0975,  0.1109,  0.1315, -0.4677,  0.2131, -0.0483,
          0.2338, -0.2709],
        [-0.1159,  0.4168, -0.2490,  0.1038,  0.1473, -0.4194,  0.0854, -0.0716,
          0.1894, -0.2436]], grad_fn=<AddmmBackward0>)

In [2]:
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)  # 输出层

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

In [3]:
net = MLP()
net(X)

tensor([[-0.0099,  0.1454,  0.2838, -0.3136,  0.0943, -0.1703, -0.0059,  0.1608,
          0.0474, -0.3629],
        [-0.0375,  0.1417,  0.1619, -0.2816,  0.0546, -0.0868,  0.0751,  0.0728,
          0.2317, -0.3503]], grad_fn=<AddmmBackward0>)

In [4]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。module的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

In [5]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.1283, -0.2505,  0.0692, -0.0353,  0.0963, -0.1223, -0.0376, -0.2937,
          0.1487,  0.2415],
        [-0.1198, -0.2287,  0.0902, -0.1079,  0.1324, -0.0883, -0.0382, -0.2606,
          0.1839,  0.2386]], grad_fn=<AddmmBackward0>)

In [6]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [8]:
net = FixedHiddenMLP()
net(X)

tensor(0.0785, grad_fn=<SumBackward0>)

In [9]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.0739, grad_fn=<SumBackward0>)

In [1]:
#5.2

In [2]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.3335],
        [-0.4271]], grad_fn=<AddmmBackward0>)

In [3]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.1782, -0.2035, -0.2761, -0.2026, -0.1256, -0.0182,  0.1783,  0.2404]])), ('bias', tensor([-0.2701]))])


In [4]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.2701], requires_grad=True)
tensor([-0.2701])


In [5]:
net[2].weight.grad == None

True

In [6]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [7]:
print(*[1])

1


In [8]:
net.state_dict()['2.bias'].data

tensor([-0.2701])

In [9]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.4647],
        [-0.4647]], grad_fn=<AddmmBackward0>)

In [10]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [11]:
rgnet[0][1][0].bias.data

tensor([-0.1368, -0.4360,  0.2891,  0.1783, -0.4031,  0.1444, -0.0729,  0.1658])

In [12]:
rgnet[0][1][0].state_dict()

OrderedDict([('weight',
              tensor([[ 0.3309, -0.2018, -0.4697, -0.4279],
                      [-0.3322, -0.1260,  0.4053,  0.3037],
                      [-0.0870,  0.1222, -0.1533,  0.4208],
                      [ 0.2295,  0.2351,  0.3565, -0.3006],
                      [ 0.1078,  0.0512,  0.3338, -0.4425],
                      [ 0.0898, -0.0208, -0.1370,  0.3823],
                      [ 0.3682, -0.1217, -0.3703,  0.0986],
                      [ 0.1293,  0.0053,  0.0513, -0.3592]])),
             ('bias',
              tensor([-0.1368, -0.4360,  0.2891,  0.1783, -0.4031,  0.1444, -0.0729,  0.1658]))])

In [13]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([0.0142, 0.0073, 0.0066, 0.0026]), tensor(0.))

In [14]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [15]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([0.3932, 0.1772, 0.6713, 0.6673])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [16]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-9.5386, -9.8476,  0.0000,  9.8381],
        [-0.0000,  0.0000,  7.8190,  5.3683]], grad_fn=<SliceBackward0>)

In [17]:
a=3

In [18]:
a*= a>10

In [19]:
net

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)

In [20]:
net[0].weight[:]

tensor([[-9.5386, -9.8476,  0.0000,  9.8381],
        [-0.0000,  0.0000,  7.8190,  5.3683],
        [ 6.7490,  0.0000,  0.0000, -0.0000],
        [ 5.0517, -9.7502,  6.4220, -0.0000],
        [ 9.5229, -9.5778, -5.8290,  0.0000],
        [ 0.0000, -0.0000,  0.0000, -0.0000],
        [ 5.9020, -0.0000, -0.0000, -5.0403],
        [-5.1756,  0.0000,  0.0000,  0.0000]], grad_fn=<SliceBackward0>)

In [21]:
net.state_dict()

OrderedDict([('0.weight',
              tensor([[-9.5386, -9.8476,  0.0000,  9.8381],
                      [-0.0000,  0.0000,  7.8190,  5.3683],
                      [ 6.7490,  0.0000,  0.0000, -0.0000],
                      [ 5.0517, -9.7502,  6.4220, -0.0000],
                      [ 9.5229, -9.5778, -5.8290,  0.0000],
                      [ 0.0000, -0.0000,  0.0000, -0.0000],
                      [ 5.9020, -0.0000, -0.0000, -5.0403],
                      [-5.1756,  0.0000,  0.0000,  0.0000]])),
             ('0.bias', tensor([0., 0., 0., 0., 0., 0., 0., 0.])),
             ('2.weight',
              tensor([[-9.6828, -0.0000, -5.2050, -0.0000,  5.3223,  9.2863, -0.0000, -7.0476]])),
             ('2.bias', tensor([0.]))])

In [22]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000, -8.8476,  1.0000, 10.8381])

In [23]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


In [24]:
#5.3

In [26]:
import torch
from torch import nn
net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(),nn.Linear(256,10))
print(net)

Sequential(
  (0): LazyLinear(in_features=0, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


/root/anaconda3/envs/d2l/lib/python3.8/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [27]:
[net[i].state_dict() for i in range(len(net))]

[OrderedDict([('weight', <UninitializedParameter>),
              ('bias', <UninitializedParameter>)]),
 OrderedDict(),
 OrderedDict([('weight',
               tensor([[ 0.0365,  0.0068,  0.0474,  ..., -0.0210, -0.0513, -0.0355],
                       [-0.0327, -0.0108, -0.0062,  ...,  0.0219,  0.0338,  0.0065],
                       [-0.0555, -0.0278,  0.0356,  ..., -0.0206, -0.0286,  0.0212],
                       ...,
                       [-0.0274,  0.0104, -0.0229,  ...,  0.0019,  0.0374, -0.0188],
                       [-0.0305,  0.0140, -0.0048,  ..., -0.0503, -0.0611,  0.0053],
                       [-0.0247, -0.0619, -0.0017,  ...,  0.0428,  0.0460,  0.0021]])),
              ('bias',
               tensor([-0.0258, -0.0467, -0.0117,  0.0112, -0.0435,  0.0133, -0.0120,  0.0402,
                        0.0445,  0.0461]))])]

In [28]:
low = torch.finfo(torch.float32).min/10
high = torch.finfo(torch.float32).max/10
X = torch.zeros([2,20],dtype=torch.float32).uniform_(low, high)
net(X)
print(net)

Sequential(
  (0): Linear(in_features=20, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


In [30]:
low

-3.4028234663852886e+37

In [31]:
high

3.4028234663852886e+37

In [1]:
#5.4

In [2]:
import torch
import torch.nn.functional as F
from torch import nn


class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

In [3]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [4]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [5]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(-6.7521e-09, grad_fn=<MeanBackward0>)

In [6]:
Y

tensor([[ 9.2394e-02, -2.4934e-01,  7.1600e-01, -1.0812e-01, -6.4436e-01,
          2.6018e-01, -1.4831e-01,  1.9070e-01, -1.3747e-01, -1.8836e-01,
          2.8194e-01,  2.8141e-01, -2.6830e-01,  5.1927e-01, -5.3656e-02,
          5.9355e-01,  7.6426e-01, -4.4146e-02, -3.7050e-01,  4.1499e-02,
          7.3881e-02, -2.3610e-01, -3.9936e-01,  1.1862e-01,  3.3051e-01,
          2.0875e-01, -4.3963e-01, -3.2148e-01, -1.8901e-01,  6.1409e-01,
          5.2177e-01, -1.5571e-01,  2.9292e-01, -6.2267e-01,  3.5871e-01,
          3.9282e-01,  1.5132e-01,  5.8188e-01, -3.0214e-01,  1.1776e-01,
         -4.7709e-01,  1.4330e-01, -8.2027e-02,  1.3908e-02,  5.0274e-01,
         -4.2411e-01, -5.0365e-01,  3.4765e-01, -9.7007e-01, -8.0717e-01,
         -4.7896e-01,  2.5106e-01, -8.3362e-02,  2.4920e-01,  1.3610e-01,
          5.8115e-01, -1.0364e-01, -1.9656e-01,  1.9953e-01,  4.1960e-01,
          4.3628e-01,  1.5992e-01, -1.8830e-01,  5.1590e-01,  3.4708e-01,
         -1.4832e-01, -5.2510e-03, -2.

In [7]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)